<a href="https://colab.research.google.com/github/savpatpanda/protein/blob/master/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [ ]:
!git clone https://github.com/savpatpanda/protein.git
%cd protein
!ls 
!pip3 install -r requirements.txt
# to push to GitHub, File > Save a copy in GitHub

#Parsing


In [ ]:
import numpy as np
from plotting import plotSpheres
def getFractionalToCartesianRotationMatrix(a, b, c, alpha, beta, gamma):
  alpha = alpha*np.pi/180
  beta  = beta*np.pi/180
  gamma  = gamma*np.pi/180

  n2 = (np.cos(alpha) - np.cos(gamma) * np.cos(beta)) / np.sin(gamma)
  M  = np.array([ [a,0,0],
                  [b*np.cos(gamma),b*np.sin(gamma),0], 
                  [c*np.cos(beta), c*n2, c*np.sqrt(np.sin(beta)**2-n2**2)]
                 ])
  return M


In [ ]:
import pandas as pd

def getResidues(pdbFile:str):
  pdb = open(pdbFile)
  data = [] ##residueCode, residueNumber, x, y, z, atomType - 2,5,6,7,8, 11
  positions = [2,5,6,7,8]

  for line in pdb:
      maxSubLength = len("ATOM    179  O   LYS A  22       7.513 -14.365  20.571")
      elements = line[:maxSubLength].split()
      if elements[0] == "ATOM" and elements[2] not in ["CA", "N", "O", "C"]:
          if len(elements) < 9:
            left = elements[2][:3]
            right = elements[2][3:]
            elements[2] = left
            elements.insert(3,right)

          item = []
          for i in positions:
            if i == 5:
              item.append(int(elements[i]))
            elif i in [6,7,8]:
              item.append(float(elements[i]))
            else:
              item.append(elements[i])
          item.append(line[-4])
            
          data.append(item)

  df = pd.DataFrame(data = data, columns = ["residueCode", "residueNumber", "x", "y", "z", "atomType"])
  df = df.groupby(by = ['residueNumber'])

  residues = []
  allProteinAtoms = []
  for key, item in df:
      atoms = []
      atomDf = df.get_group(key)
      for index, row in atomDf.iterrows():
        atoms.append(Atom(row["x"], row["y"], row["z"], row["atomType"], row["residueCode"], charge = 0))
      residues.append(Residue(atoms, atomDf["x"].mean(), atomDf["y"].mean(), atomDf["z"].mean()))
      allProteinAtoms.extend(atoms)
  return allProteinAtoms, residues

In [ ]:
from gemmi import cif 
%matplotlib inline
import matplotlib.pyplot as plt

parseCIFNum = lambda num: float(num[:-4] if num[-1] == ')' else num)

def parseCIFFile(cifFileName):
  doc = cif.read_file("./data/185472.cif")
  block = doc.sole_block()    
  fractionalSpace = [ block.find_pair("_cell_length_a"),
                      block.find_pair("_cell_length_b"),
                      block.find_pair("_cell_length_c"),
                      block.find_pair("_cell_angle_alpha"),
                      block.find_pair("_cell_angle_beta"),
                      block.find_pair("_cell_angle_gamma") ]

  fractionalSpace = map(lambda x: parseCIFNum(x[1]), fractionalSpace)
  rotationMatrix = getFractionalToCartesianRotationMatrix(*fractionalSpace)
  
  x = [parseCIFNum(x) for x in list(block.find_loop("_atom_site_fract_x"))]
  y = [parseCIFNum(x) for x in list(block.find_loop("_atom_site_fract_y"))]
  z = [parseCIFNum(x) for x in list(block.find_loop("_atom_site_fract_z"))]
  coords = np.array(list(zip(x,y,z)))

  return coords, rotationMatrix
  
try:
  coords, rotationMatrix = parseCIFFile("./data/185472.cif")
  cartesian = np.matmul(coords, rotationMatrix)
  fig = plt.figure()
  ax = fig.add_subplot(111, projection='3d')
  print(cartesian)
  plotSpheres(cartesian*30, ax)
  plt.show()
  
except Exception as e:
  print(e)

# Data Structures

In [ ]:
class Residue():
  def __init__ (self, atoms, meanX, meanY, meanZ):
    self.atoms = atoms
    self.position = (meanX, meanY, meanZ)


class Atom():
  def __init__(self, x, y, z, atomType, residueComponents = None, charge = 0):
    self.position = (x,y,z)
    self.atomType = atomType
    self.residueComponents = residueComponents
    self.charge = charge
    
  def __repr__(self):
    return f"{self.atomType} at {self.position}"

class Compound():
  def __init__(self, atoms):
    self.atoms = atoms
    self.positions = [a.position for a in self.atoms]
    separated = list(zip(*self.positions))
    self.mean = list(map(lambda s: np.array(s).mean(), separated))

  def translate(self, dx, dy, dz):
    return list(map(lambda p: (p[0]+dx,p[1]+dy,p[2]+dz), self.positions))

def plotAtoms(ax, atoms: list, color: str, radius: float):
  plotSpheres(atoms, ax, radius = radius, color = color)

In [ ]:
coords, rotationMatrix = parseCIFFile("./data/185472.cif")
atoms, residues = getResidues('data/1ikf.pdb')
cartesian = np.matmul(coords, rotationMatrix)
fig = plt.figure()
fig.set_size_inches(10, 10)
ax = fig.add_subplot(111, projection='3d')
plotAtoms(ax, cartesian, color = "green", radius = 0.2)
plt.show()
fig = plt.figure()
fig.set_size_inches(10, 10)
ax = fig.add_subplot(111, projection='3d')
plotAtoms(ax, [atom.position for atom in atoms], color = "red", radius = 0.2)


In [ ]:
wellCoeff = {
    "C" : 0.15, 
    "N" : 0.16,
    "O" : 0.2,
    "S" : 0.2,
    "H" : 0.02,
    "P" : 0.36758504
} #kcal / mol

atomicRadii = {
    "C" : 70, 
    "N" : 65,
    "O" : 60,
    "S" : 100,
    "H" : 25,
    "P" : 98
} #picometers

def LJ(type1, type2, distance):
  rij_min = (atomicRadii[type1] + atomicRadii[type2])/2
  ljEnergy = math.sqrt(wellCoeff[type1]*wellCoeff[type2]) * ((rij_min/distance)**12 - 2*(rij_min/distance)**6)
  return ljEnergy

def electro(atom1, atom2, distance):
  cutoffCharge = 1 
  dielectric = 8.987551 * (10**9) / 80.1 #H20: https://chem.libretexts.org/Bookshelves/Analytical_Chemistry/Book%3A_Physical_Methods_in_Chemistry_and_Nano_Science_(Barron)/02%3A_Physical_and_Thermal_Analysis/2.09%3A_Electrical_Permittivity_Characterization_of_Aqueous_Solutions
  if atom1.charge < cutoffCharge or atom2.charge < cutoffCharge:
    return 0
  else:
    electroEnergy = (atom1.charge) * (atom2.charge) / (dielectric * distance)
    return electroEnergy

def potentialEnergy(proteinSet, compoundSet):
  totalEnergy = 0
  cutOffDistance = 1e9
  for i in proteinSet:
    for j in compoundSet:
      distance = math.sqrt( (i.position[0] - j.position[0])**2 + (i.position[1] - j.position[1])**2 + (i.position[2] - j.position[2])**2)
      if distance > cutOffDistance: #optional cutoff for particles too far away
        continue
      totalEnergy += (LJ(i.atomType, j.atomType, distance) + electro(i.atomType, j.atomType, distance))
  return totalEnergy

    

In [ ]:
from proteinModel import rotate, rotationMatrix
from itertools import combinations

%matplotlib inline

def boundingBox(atoms):
  minX, maxX = float('inf'), float('-inf')
  minY, maxY = float('inf'), float('-inf')
  minZ, maxZ = float('inf'), float('-inf')
  for atom in atoms:
    x,y,z = atom
    minX, maxX = min(minX, x), max(maxX, x)
    minY, maxY = min(minY, y), max(maxY, y)
    minZ, maxZ = min(minZ, z), max(maxZ, z)
  boundbox = []
  for x in [minX, maxX]:
    for y in [minY, maxY]:
      for z in [minZ, maxZ]:
        boundbox.append((x,y,z))
  volume = (maxX - minX) * (maxY - minY) * (maxZ - minZ)
  return boundbox, volume

def findMinBoundingBox(atoms, getPosition, rotateX, rotateY, rotateZ):
  deltas = range(0, 90, 5)
  smallestVolume, smallestBoundingBox, opt = float('inf'), None, None
  finalAtoms = None
  for dx,dy,dz in combinations(deltas, 3):
    vector = [getPosition(a) for a in atoms]
    atomSet = rotate(vector, rotationMatrix(dx, dy, dz))
    boundbox, volume = boundingBox(atomSet)
    if volume < smallestVolume:
      smallestVolume = volume
      smallestBoundingBox = boundbox
      opt = (dx, dy, dz)
      finalAtoms = atomSet
    
  return smallestVolume, smallestBoundingBox, opt, finalAtoms

def testBoundingBox():
  atoms, residues = getResidues('data/1ikf.pdb')
  _, bb, _, finalAtoms = findMinBoundingBox(atoms, lambda x : x.position, 30, 0, 0)
  fig = plt.figure()
  fig.set_size_inches(10, 10)
  ax = fig.add_subplot(111, projection='3d')
  plotAtoms(ax, finalAtoms, color = "green", radius = 0.2)
  plotAtoms(ax, bb, color = "red", radius = 0.3)
  plt.show()

# Optimization Ideas
- Gradient Search
  - Bounding box optimization
  - Translate, rotate, translate
  - Informed search using important residues (e.g Euclidean distances)
  - How to descend:
    - PyTorch autograd. The idea is to convert our energy functions into matrix operations that take a central position of the compound, and use our resultant energy as loss. We can then use gradients from these computations to nudge our central position in the correct direction.
    - Beam search - from an initial position, compute different directions to translate and their resultant energies, and pick the locally optimal one, continue until you find the min.
- Residue / Active site pockets
    - Move the chemical compound along the normal of the plane in the pocket.


In [ ]:
import torch
#https://pytorch.org/tutorials/beginner/pytorch_with_examples.html
dtype = torch.float
device = torch.device("cpu")
epochs = 1000
learning_rate = 1e-3
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# Create random Tensors for weights.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
w1 = torch.randn(1, 3, device=device, dtype=dtype, requires_grad=True)
print(w1)

for t in range(epochs + 1):
    # Forward pass: compute predicted y using operations on Tensors; these
    # are exactly the same operations we used to compute the forward pass using
    # Tensors, but we do not need to keep references to intermediate values since
    # we are not implementing the backward pass by hand.
    '''
    Question: Is it possible to create these operations with matrix manipulation --
    w1 -> translated compound
    translated compound -> LJ 
    translated compound -> Electric Energies
    '''
    y_pred = w1 * torch.ones(3)

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    loss = (y_pred - torch.tensor([5,5,5])).pow(2).sum()
    print(t, loss.item())

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call w1.grad and w2.grad will be Tensors holding the gradient
    # of the loss with respect to w1 and w2 respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    # An alternative way is to operate on weight.data and weight.grad.data.
    # Recall that tensor.data gives a tensor that shares the storage with
    # tensor, but doesn't track history.
    # You can also use torch.optim.SGD to achieve this.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad

        # Manually zero the gradients after updating weights
        w1.grad.zero_()
print(w1)  
